In [1]:
# import sys
# !{sys.executable} -m pip install python_dotenv
# !{sys.executable} -m pip install python_binance
import os
from binance.client import Client
from binance.spot import Spot

import pandas as pd
from dotenv import load_dotenv


In [2]:
# read from local .env file
load_dotenv() 

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_API_SECRET']

client = Client()
spotClient = Spot()
# myClient = Client(api_key, api_secret, testnet=True)

In [3]:
# INPUT
## SET this to get d2, d3, d4
day_aggregation = 10

## ema (20 or 25)
ema_size = 25

## A symbol with streak >= this will be printed out
streak_threshold = 20

## how many day to query
quantity = 6000

quote_asset = "BTC"


In [4]:
# TIMEFRAME
## set timeframe to query binance, based on day_aggregation
do_manual_aggregation = True
timeframe = '1d'
if day_aggregation == 3:
    timeframe = '3d'
    do_manual_aggregation = False
elif day_aggregation == 7:
    timeframe = '1w'
    do_manual_aggregation = False
elif day_aggregation == 30:
    timeframe = '1m'
    do_manual_aggregation = False

In [5]:
# FUNCTIONS
## Function to query Binance for candlestick data
def get_candlestick_data(symbol, timeframe, qty):
    # Retrieve the raw data
    raw_data = spotClient.klines(symbol=symbol, interval=timeframe, limit=qty)
    # Set up the return array
    converted_data = []
    # Convert each element into a Python dictionary object, then add to converted_data
    for candle in raw_data:
        # Dictionary object
        converted_candle = {
            'time': candle[0],
            'open': float(candle[1]),
            'high': float(candle[2]),
            'low': float(candle[3]),
            'close': float(candle[4]),
            'volume': float(candle[5]),
            'close_time': candle[6],
            'quote_asset_volume': float(candle[7]),
            'number_of_trades': int(candle[8]),
            'taker_buy_base_asset_volume': float(candle[9]),
            'taker_buy_quote_asset_volume': float(candle[10])
        }
        # Add to converted_data
        converted_data.append(converted_candle)
    # Return converted data
    return converted_data

## Define function to calculate an arbitrary EMA      
def calc_generic_ema(dataframe, symbol, timeframe, ema_size):
    # Create column string
    ema_name = "ema_" + str(ema_size)
    # Create the multiplier
    multiplier = 2/(ema_size + 1)
    # Calculate the initial value (SMA)
    # pd.set_option('display.max_columns', None) # <- use this to show all columns
    # pd.set_option('display.max_rows', None) # <- use this to show all the rows
    initial_mean = dataframe['close'].head(ema_size).mean()

    # Iterate through Dataframe
    for i in range(len(dataframe)):
        if i == ema_size:
            dataframe.loc[i, ema_name] = initial_mean
        elif i > ema_size:
            ema_value = dataframe.loc[i, 'close'] * multiplier + dataframe.loc[i-1, ema_name]*(1-multiplier)
            dataframe.loc[i, ema_name] = ema_value
        else:
            dataframe.loc[i, ema_name] = 0.00
    # print(dataframe) # <- use this to print the dataframe if you want to inspect
    return dataframe

In [6]:
# GET SYMBOL LIST
info = client.get_exchange_info()
coinlist = []
for c in info['symbols']:
    if c['quoteAsset']==quote_asset and c['status']=="TRADING":
        coinlist.append(c['symbol'])
        
print(len(coinlist))
sublist = coinlist[0:2]
# sublist = coinlist
print('Scanning ' + str(len(sublist)) + ' trading pairs...')

283
Scanning 2 trading pairs...


In [7]:
# PROGRESS BAR
from ipywidgets import IntProgress
from IPython.display import display
import time

max_count = len(sublist)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

count = 0

# MAIN
for idx, symbol in enumerate(sublist):
    # update progress bar
    f.value += 1
    
    raw_data = get_candlestick_data(symbol=symbol, timeframe=timeframe, qty=1000)
    ## Convert into Dataframe
    df = pd.DataFrame(raw_data)

    if do_manual_aggregation:
        df['d_index'] = df.index//day_aggregation
        # get high / low
        df['d_high'] = df.groupby('d_index')['high'].transform(max)
        df['d_low'] = df.groupby(['d_index'])['low'].transform(min)

        # get open / close

        df['d_open'] = df.groupby(['d_index'])['open'].transform('first')
        df['d_close'] = df.groupby(['d_index'])['close'].transform('last')

        df = df.groupby('d_index').first()

        # clean up d1 column
        del df['open']
        del df['high']
        del df['low']
        del df['close']

        # rename d_column
        df = df.rename(columns={"d_high": "high", "d_low": "low", "d_close": "close", "d_open": "open" })

    df['time'] = pd.to_datetime(df['time'], unit='ms')
    # ema
    df = calc_generic_ema(df, symbol, timeframe, ema_size)

    # clean up

    del df['volume']
    del df['close_time']
    del df['quote_asset_volume']
    del df['number_of_trades']
    del df['taker_buy_base_asset_volume']
    del df['taker_buy_quote_asset_volume']

    # Calculate oversell
    df['oversell_day'] = 0
    df.loc[df['high'] > df['ema_25'], 'oversell_day'] = 1
    df.loc[df['ema_25'] == 0, 'oversell_day'] = 0


    df['streak'] = df.groupby((df["oversell_day"] != df["oversell_day"].shift()).cumsum()).cumcount()+1
    df.loc[df['oversell_day'] == 0, 'streak'] = 0
    
#     if (df['streak'].iloc[-1] > streak_threshold):
    print('Symbol: '+ symbol + ' is having an oversell streak')
    display(df['streak'].iloc[-1])
    display(df.tail(30))
        
    if f.value == max_count:
        print('Scan Completed!')

IntProgress(value=0, max=2)

Symbol: ETHBTC is having an oversell streak


0

,time,high,low,open,close,ema_25,oversell_day,streak
d_index,,,,,,,,
70,2022-06-06,0.061011,0.049081,0.060368,0.054807,0.067712,0,0
71,2022-06-16,0.058511,0.049916,0.054806,0.057796,0.066949,0,0
72,2022-06-26,0.059159,0.053069,0.057797,0.056143,0.066118,0,0
73,2022-07-06,0.060717,0.053202,0.056142,0.059105,0.065579,0,0
74,2022-07-16,0.072331,0.057946,0.059106,0.067604,0.065735,1,1
75,2022-07-26,0.073902,0.065136,0.067601,0.071080,0.066146,1,2
76,2022-08-05,0.081653,0.070984,0.071081,0.079619,0.067182,1,3
77,2022-08-15,0.080970,0.073126,0.079619,0.077528,0.067978,1,4
78,2022-08-25,0.080766,0.072737,0.077522,0.078530,0.068790,1,5


Symbol: LTCBTC is having an oversell streak


0

,time,high,low,open,close,ema_25,oversell_day,streak
d_index,,,,,,,,
70,2022-06-06,0.002254,0.001716,0.002110,0.002243,0.002850,0,0
71,2022-06-16,0.002801,0.002127,0.002242,0.002739,0.002841,0,0
72,2022-06-26,0.002824,0.002448,0.002740,0.002463,0.002812,1,1
73,2022-07-06,0.002597,0.002360,0.002462,0.002485,0.002787,0,0
74,2022-07-16,0.002741,0.002432,0.002486,0.002534,0.002768,0,0
75,2022-07-26,0.002698,0.002487,0.002534,0.002637,0.002758,0,0
76,2022-08-05,0.002687,0.002515,0.002637,0.002597,0.002745,0,0
77,2022-08-15,0.002692,0.002496,0.002599,0.002642,0.002737,0,0
78,2022-08-25,0.003120,0.002583,0.002641,0.003034,0.002760,1,1


Scan Completed!
